In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import csv

def crawl_tiki_reviews_selenium(product_url, max_scrolls=5):
    # Setup trình duyệt
    options = Options()
    options.add_argument('--headless')  # chạy ẩn không hiển thị cửa sổ
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get(product_url)
    time.sleep(3)  # đợi trang load

    # Cuộn xuống để load đánh giá
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Dùng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    reviews = []
    review_elements = soup.select('div.review')  # selector có thể thay đổi theo DOM

    for r in review_elements:
        try:
            name = r.select_one('.user').get_text(strip=True)
            rating = len(r.select('.icon-star--gold'))
            content = r.select_one('.review-comment__content').get_text(strip=True)
            date = r.select_one('.review-comment__date').get_text(strip=True)

            reviews.append({
                'name': name,
                'rating': rating,
                'content': content,
                'date': date
            })
        except:
            continue

    return reviews

# Ví dụ: Crawl đánh giá từ một sản phẩm
product_url = "https://tiki.vn/may-lanh-lg-1-hp-inverter-v10apf-p181692647.html"
reviews = crawl_tiki_reviews_selenium(product_url)

# In kết quả
for i, r in enumerate(reviews[:3], 1):
    print(f"\n--- Review {i} ---")
    print(f"Tên: {r['name']}")
    print(f"Sao: {r['rating']}")
    print(f"Nội dung: {r['content']}")
    print(f"Ngày: {r['date']}")
